In [5]:
import fasttext
import numpy as np
import sys
sys.path.append('..')

In [2]:
import logging
log_format = '[%(asctime)s] [%(levelname)s] - %(message)s'
logging.basicConfig(level=logging.DEBUG, format=log_format)
log = logging.getLogger(__name__)

# writing to stdout
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
handler.setFormatter(log_format)
log.addHandler(handler)
logging.debug('hello')

[2020-01-11 00:33:12,522] [DEBUG] - hello


In [3]:
model = fasttext.load_model('/home/legin/fasttext/wiki.en.bin')

In [6]:
import compressors.dumb as d 
import storages.dict_based as db 
from importlib import reload
reload(d)

<module 'compressors.dumb' from '/home/legin/ft_compress/ft_compress/compressors/dumb.py'>

In [7]:
dc = d.DumbCompressor(db.DictStorage()) #~50% percents

In [9]:
dc.fit(model)

[2020-01-11 00:34:14,307] [DEBUG] - loading words
[2020-01-11 00:34:38,677] [DEBUG] - filling matrix


TypeError: 'numpy.int64' object is not iterable

In [12]:
#del dc
import gc 
gc.collect()

3299

In [4]:
import compressors.compressor8 as d 
from importlib import reload
reload(d)

<module 'compressors.compressor8' from '/home/legin/ft_compress/ft_compress/compressors/compressor8.py'>

In [5]:
dc = d.Compressor8bit(model, 2500000) #~50% percents

[2020-01-04 23:04:07,559] [DEBUG] - Loading words
[2020-01-04 23:04:16,689] [WARNING] - Too many ngrams, breaking
[2020-01-04 23:04:16,690] [WARNING] - Too many ngrams, breaking
[2020-01-04 23:04:16,690] [DEBUG] - Estimating parameters
[2020-01-04 23:04:37,513] [DEBUG] - filling matrix
[2020-01-04 23:05:07,033] [DEBUG] - ready


In [103]:
np.fromstring(dc.ngrams[','], np.uint8)

/home/legin/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """Entry point for launching an IPython kernel.


array([133,  67,  94,  53, 127, 134, 112,  69,  98, 151,  69,  78, 130,
       110,  98,  59,  55,  41, 106, 118, 134, 143,  88, 105,  95, 109,
       103, 103, 121, 134, 100,  37,  69, 113, 110, 132,  54,  67,  60,
       119, 103, 136,  80,  76,  52,  42,  33,  57,  66,  88, 103,  89,
       116, 166, 170,  92, 113, 117, 109,  83, 119,  71, 145, 130,  29,
       138, 138,  75, 114,  96,  90,  79,  43, 130, 181,  83,  13,  27,
        63, 102,  21, 110, 148, 106, 144,  73,  99,   0,  92,  82, 156,
       120,  63, 168,  49, 115,  45, 103,  93, 114,  91, 139,  72, 102,
        61, 151,  76,  65, 151,  97,  43, 144, 157,  52,  25, 154,  57,
        79,  73,  36,  28,  90, 151,  95,  45,  55,  91,  59, 124,  41,
       106, 141, 207,  87, 107,  92, 154,  73, 129,  90,  89,  54,  12,
        39, 111,  80, 182, 124,  71,  37,  63,  82, 151, 121, 100, 114,
        73,  94, 117, 153, 105, 101,  56, 114, 139, 111,  93, 158,  97,
        90,  85, 110, 115,  60,  95,  46,  74,  70,  74, 145,  5

In [8]:
v = model.get_input_vector(0)

In [105]:
v2 = np.round(255*(v-dc.min_)/(dc.max_-dc.min_)).astype(np.uint8)


In [10]:
(dc.bytes_to_vec(dc.ngrams[',']) - v).sum()

-0.14907378516543712

In [101]:
dc.max_, v.max()

(3.0360308, 3.0360308)

In [92]:
dc.ngrams

{',': b"\x85C^5\x7f\x86pEb\x97EN\x82nb;7)jv\x86\x8fXi_mggy\x86d%Eqn\x846C<wg\x88PL4*!9BXgYt\xa6\xaa\\qumSwG\x91\x82\x1d\x8a\x8aKr`ZO+\x82\xb5S\r\x1b?f\x15n\x94j\x90Ic\x00\\R\x9cx?\xa81s-g]r[\x8bHf=\x97LA\x97a+\x90\x9d4\x19\x9a9OI$\x1cZ\x97_-7[;|)j\x8d\xcfWk\\\x9aI\x81ZY6\x0c'oP\xb6|G%?R\x97ydrI^u\x99ie8r\x8bo]\x9eaZUns<_.JFJ\x91;\x94+}v\x82K\x17\x0eV:\x98\x01Gdivo\xa2m;}\x15Gyn}XX\x0f!=\x07;7>\x8fiUd\xa3b!+`CTFR\x82\xb0tZU0\x0e@`\x95Un\xab89\xaeL\x8bph\xb9]\x92kQ\x0f=tuhlo7_i\xa5\xffqnNR\x9fje0}o[-e}WY^\x82Et\x7f\xa3UVaqbO\xa29\x88>E'"}

In [6]:
dc.info()

'Number of ngrams: 2500000. Approximate size: 729.391224861145 MB'